# Machine Learning in Python - Deep Learning

Create a simple feed forward network using keras

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display, HTML, Image
from IPython.display import SVG

import matplotlib.pyplot as plt
%matplotlib inline
#plt.style.use('fivethirtyeight')

from TAS_Python_Utilities import data_viz
from TAS_Python_Utilities import visualize_tree

from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 

from keras.models import Sequential
from keras.layers import Dense, Activation, Merge, Dropout
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import model_to_dot


ModuleNotFoundError: No module named 'TAS_Python_Utilities'

## Dataset Pre-Processing

Read in a dataset

In [ ]:
abt = pd.read_csv("mnist_train_small.csv", encoding = "ISO-8859-1")

# Put all but the target variable into the descriptive features array
X = abt[abt.columns.difference(["value"])]
Y = abt["value"]

# Use a range scaling to scale all variables to between 0 and 1
min_max_scaler = preprocessing.MinMaxScaler()
cols = X.columns
X = pd.DataFrame(min_max_scaler.fit_transform(X), columns = cols) # Watch out for putting back in columns here

X_train_plus_valid, X_test, y_train_plus_valid, y_test = train_test_split(X, Y, random_state=0, test_size = 0.30, train_size = 0.7)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_plus_valid, y_train_plus_valid, random_state=0, test_size = 0.199/0.7, train_size = 0.5/0.7)

In [ ]:
X_train_plus_valid.shape

In [ ]:
abt.head()

In [ ]:
# Convert the singl column label into a dummy coded label
y_train_wide = to_categorical(np.asarray(y_train))
y_valid_wide = to_categorical(np.asarray(y_valid))

In [ ]:
y_train_wide

## Model Building

Specfiy the structure of the neural network model

In [ ]:
model = Sequential()
model.add(Dense(input_dim=784, units=512))
model.add(Activation("sigmoid"))
model.add(Dense(units=207))
model.add(Activation("sigmoid"))
model.add(Dense(units=102))
model.add(Activation("sigmoid"))
model.add(Dense(units=10))
model.add(Activation("softmax"))
model.summary()

Draw the model

In [ ]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

Specify the parameters of the model optimisation process

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Train the model

In [ ]:
history = model.fit(np.asfarray(X_train), np.asfarray(y_train_wide), \
          epochs=20, batch_size=32, verbose=1, \
          validation_data=(np.asfarray(X_valid), np.asfarray(y_valid_wide)))

Plot learning curves showing accuracy

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.ylim((0, 1.1))
plt.show()

Plot learning curves showing loss

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.ylim((-0.1, 1.1))
plt.show()

Evaluate the model

In [ ]:
print("******** Training Data ********")
# Make a set of predictions for the training data
y_pred = model.predict_classes(np.asfarray(X_train), batch_size=32)

# Print performance details
print(metrics.classification_report(y_train, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_train, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))



print("****** Validation Data ********")

# Make a set of predictions for the validation data
y_pred = model.predict_classes(np.asfarray(X_valid))

# Print performance details
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_valid, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))

print("****** Test Data ********")

# Make a set of predictions for the validation data
y_pred = model.predict_classes(np.asfarray(X_test))

# Print performance details
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))

## Building a Better Model

A more sophisticated model - based on [https://raw.githubusercontent.com/fchollet/keras/master/examples/mnist_mlp.py](https://raw.githubusercontent.com/fchollet/keras/master/examples/mnist_mlp.py)

In [ ]:
model_mlp = Sequential()
model_mlp.add(Dense(512, input_shape=(784,)))
model_mlp.add(Activation('relu'))
model_mlp.add(Dropout(0.2))
model_mlp.add(Dense(512))
model_mlp.add(Activation('relu'))
model_mlp.add(Dropout(0.2))
model_mlp.add(Dense(10))
model_mlp.add(Activation('softmax'))
model_mlp.summary()

Draw the model

In [ ]:
SVG(model_to_dot(model_mlp).create(prog='dot', format='svg'))

Train the model

In [ ]:
model_mlp.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Convert the singl column label into a dummy coded label
history_mlp = model_mlp.fit(np.asfarray(X_train), np.asfarray(y_train_wide), \
          epochs=20, batch_size=32, verbose=1, \
          validation_data=(np.asfarray(X_valid), np.asfarray(y_valid_wide)))

Plot learning curves showing accuracy

In [ ]:
# summarize history for accuracy
plt.plot(history_mlp.history['acc'])
plt.plot(history_mlp.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.ylim((0, 1.1))
plt.show()

Plot learning curves showing loss

In [ ]:
# summarize history for loss
plt.plot(history_mlp.history['loss'])
plt.plot(history_mlp.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.ylim((-0.1, 1.1))
plt.show()

Evaluate the model

In [ ]:
print("******** Training Data ********")
# Make a set of predictions for the training data
y_pred = model_mlp.predict_classes(np.asfarray(X_train), batch_size=32)

# Print performance details
print(metrics.classification_report(y_train, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_train, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))



print("****** Validation Data ********")

# Make a set of predictions for the validation data
y_pred = model_mlp.predict_classes(np.asfarray(X_valid))

# Print performance details
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_valid, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))




print("****** Test Data ********")

# Make a set of predictions for the validation data
y_pred = model_mlp.predict_classes(np.asfarray(X_test))

# Print performance details
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))